<a href="https://colab.research.google.com/github/valentinocc/Keras_cifar10/blob/master/cifar10_ELUcnn_tpu_BROKEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed May 29 10:42:05 2019

@author: val

basic CNN for cifar10 dataset
"""
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
%load_ext tensorboard.notebook

In [0]:
def model(input_shape):

    model = tf.keras.models.Sequential()
    convolution_elu_block(model, 32, (3, 3), 'same', input_shape = input_shape, first_layer = True)
    convolution_elu_block(model, 32, (3, 3), 'same')
    model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    convolution_elu_block(model, 64, (3, 3), 'same')
    convolution_elu_block(model, 64, (3, 3), 'same')
    model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    
    model.add(tf.keras.layers.Flatten())
    dense_elu_block(model, 256)
    final_dense_layer(model, 10)
    
    model.summary()
    
    return model

def convolution_elu_block(model, filters, filter_shape, padding_setting, input_shape = None, first_layer = False):

    if (first_layer):
        model.add(tf.keras.layers.Conv2D(filters, filter_shape, padding = padding_setting, data_format = 'channels_last', input_shape = input_shape))
    else:
        model.add(tf.keras.layers.Conv2D(filters, filter_shape, padding = padding_setting))
    
    model.add(tf.keras.layers.Activation('elu'))

def dense_elu_block(model, units):
    
    model.add(tf.keras.layers.Dense(units))
    model.add(tf.keras.layers.Activation('elu'))
    
def final_dense_layer(model, classes_amount):
    model.add(tf.keras.layers.Dense(classes_amount))
    model.add(tf.keras.layers.Activation('softmax'))
    

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.006*8, shuffle= True)


170500096/170498071 [==============================] - 2s 0us/step


In [8]:
input_shape = x_train.shape[1:]
keras_model = model(input_shape)

#create TensorBoard callback
logdir="logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tpu_model = tf.contrib.tpu.keras_to_tpu_model(keras_model, strategy = tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(tpu = TPU_WORKER)))
tpu_model.compile( optimizer = tf.train.AdamOptimizer(learning_rate = 1e-3), loss = tf.keras.losses.sparse_categorical_crossentropy, metrics = ['sparse_categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 16, 16, 64)        0         
__________

In [9]:
model_convergence_data = tpu_model.fit(x_train, y_train, epochs = 20, batch_size = 32*8, validation_data = (x_valid, y_valid), verbose=1, callbacks=[tensorboard_callback])
tpu_model.evaluate(x_test, y_test, batch_size = 32*8)
tpu_model.save_weights('./tpu_model_weights.h5', overwrite = True)

Train on 47600 samples, validate on 2400 samples
Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(32,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(32, 32, 32, 3), dtype=tf.float32, name='conv2d_4_input_10'), TensorSpec(shape=(32, 1), dtype=tf.int32, name='activation_9_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for conv2d_4_input
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 3.2093029022216797 secs
INFO:tensorflow:Setting weights on TPU model.
47104/47600 [============================>.] - ETA: 0s - loss: 14.4948 - sparse_categorical_accuracy: 0.0998INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(30,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(30, 32, 32, 3), dtype=tf.float32, name='conv2d_4_input_10'), TensorSpec(shape=(30,

KeyboardInterrupt: ignored

In [0]:
%tensorboard --logdir logs/scalars